In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.layered.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.lqns.*
import jline.solvers.ln.*

In [ ]:
// Set verbose level
GlobalConstants.setVerbose(VerboseLevel.SILENT)

println("This example illustrates a layered network with a loop.")

// Create model
val model = LayeredNetwork("myLayeredModel")

// Layer 1 - Reference task
val P1 = Processor(model, "P1", GlobalConstants.MaxInt.toDouble(), SchedStrategy.INF)
val T1 = Task(model, "T1", 1, SchedStrategy.REF).on(P1)
T1.setThinkTime(Immediate())
val E1 = Entry(model, "Entry").on(T1)

// Layer 2 - Service task
val P2 = Processor(model, "P2", GlobalConstants.MaxInt.toDouble(), SchedStrategy.INF)
val T2 = Task(model, "T2", 1, SchedStrategy.INF).on(P2).setThinkTime(Immediate())
val E2 = Entry(model, "E2").on(T2)

// Layer 3 - Backend task
val P3 = Processor(model, "P3", 5, SchedStrategy.PS)
val T3 = Task(model, "T3", 20, SchedStrategy.INF).on(P3)
T3.setThinkTime(Exp.fitMean(10.0))
val E3 = Entry(model, "E1").on(T3) // Note: Entry named 'E1' in original

In [ ]:
// Activities for T1 (with loop)
val A1 = Activity(model, "A1", Exp.fitMean(1.0)).on(T1).boundTo(E1)
val A2 = Activity(model, "A2", Exp.fitMean(2.0)).on(T1)
val A3 = Activity(model, "A3", Exp.fitMean(3.0)).on(T1).synchCall(E2)

// Activities for T2 (with fork-join)
val B1 = Activity(model, "B1", Exp.fitMean(0.1)).on(T2).boundTo(E2)
val B2 = Activity(model, "B2", Exp.fitMean(0.2)).on(T2)
val B3 = Activity(model, "B3", Exp.fitMean(0.3)).on(T2)
val B4 = Activity(model, "B4", Exp.fitMean(0.4)).on(T2)
val B5 = Activity(model, "B5", Exp.fitMean(0.5)).on(T2)
val B6 = Activity(model, "B6", Exp.fitMean(0.6)).on(T2).synchCall(E3).repliesTo(E2)

// Activities for T3 (with or-fork/join)
val C1 = Activity(model, "C1", Exp.fitMean(0.1)).on(T3).boundTo(E3)
val C2 = Activity(model, "C2", Exp.fitMean(0.2)).on(T3)
val C3 = Activity(model, "C3", Exp.fitMean(0.3)).on(T3)
val C4 = Activity(model, "C4", Exp.fitMean(0.4)).on(T3)
val C5 = Activity(model, "C5", Exp.fitMean(0.5)).on(T3).repliesTo(E3)

In [ ]:
// Add precedence relationships

// T1: Loop with 3 iterations
T1.addPrecedence(ActivityPrecedence.Loop(A1, arrayOf(A2, A3), 3))

// T2: Serial connection and AND fork-join
T2.addPrecedence(ActivityPrecedence.Serial(B4, B5))
T2.addPrecedence(ActivityPrecedence.AndFork(B1, arrayOf(B2, B3, B4)))
T2.addPrecedence(ActivityPrecedence.AndJoin(arrayOf(B2, B3, B5), B6))

// T3: OR fork-join with probabilities
T3.addPrecedence(ActivityPrecedence.OrFork(C1, arrayOf(C2, C3, C4), doubleArrayOf(0.3, 0.3, 0.4)))
T3.addPrecedence(ActivityPrecedence.OrJoin(arrayOf(C2, C3, C4), C5))

In [ ]:
// Solve with different solvers

// LQNS solver
if (LQNS.isAvailable()) {
    try {
        val solverLqns = LQNS(model)
        val avgTableLqns = solverLqns.avgTable
        println("LQNS Solver Results:")
        avgTableLqns.print()
    } catch (e: Exception) {
        println("LQNS solver error: ${e.message}")
    }
} else {
    println("LQNS solver not available")
}

// LN solver
try {
    val solverLn = LN(model)
    val avgTableLn = solverLn.avgTable
    println("\nLN Solver Results:")
    avgTableLn.print()
} catch (e: Exception) {
    println("LN solver error: ${e.message}")
}